# RASPA Agent Playground

This notebook demonstrates the `RaspaAgent` which converts natural language into valid `simulation.input` files for molecular simulations.

### Setup
First, we install the necessary dependencies.

### Import the Agent
Make sure `raspa_agent.py` is in the same folder as this notebook.

In [2]:
import importlib
import sys

# Force reload of raspa_agent to get latest code
if 'raspa_agent' in sys.modules:
    importlib.reload(sys.modules['raspa_agent'])
    print("✔ Reloaded raspa_agent module")

from raspa_agent import RaspaAgent
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

agent = RaspaAgent(api_key=GOOGLE_API_KEY)

# Verify which model is being used
if hasattr(agent, 'model'):
    print(f"✔ Using model: {agent.model.model_name if hasattr(agent.model, 'model_name') else 'gemini-2.5-flash'}")
else:
    print("Running in mock mode (no API key)")


✔ Using model: models/gemini-2.5-flash


/Users/harshit/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/harshit/Desktop/ig_6th/UGP/CODE/WEEK1/raspa_agent.py:71: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


### Scenario 1: Simple Request (Defaults Heavy)
User asks for a framework but specifies nothing else. The agent should fill in Temperature (298K), Pressure (1e5 Pa), and standard Monte Carlo parameters.

In [5]:
request_1 = "Simulate water in UIO-66 at 300K and 1bar ."

# If you don't have an API Key, set mock=True to see the logic flow
output_1 = agent.run(request_1, mock=(GOOGLE_API_KEY is None), output_filename="simulation_case1.input")

print("\n--- Generated File Preview ---\n")
print(output_1)

--- Processing Request: 'Simulate water in UIO-66 at 300K and 1bar .' ---
[Request ID: 5]
✔ Extracted Entities: {'simulation_type': 'MonteCarlo', 'system': {'framework_name': 'UIO-66', 'temperature': 300.0, 'pressure_list': [100000.0]}, 'parameters': {}, 'component': {'name': 'water', 'mole_fraction': 1.0}}
✔ Applied Defaults.
✔ RASPA file generated (1083 chars)
✔ File saved to: simulation_case1.input

--- Generated File Preview ---

SimulationType                MonteCarlo
NumberOfCycles                25000
NumberOfInitializationCycles  5000
PrintEvery                    1000
RestartFile                   no

Forcefield                    UFF
CutOff                        12.0
ChargeMethod                  Ewald
EwaldPrecision                1e-06
RemoveAtomNumberCodeFromLabel yes

Framework 0
FrameworkName UIO-66
UnitCells 2 2 2
HeliumVoidFraction 0.29
ExternalTemperature 300.0
ExternalPressure 100000.0

ComputeNumberOfMoleculesHistogram yes
WriteNumberOfMoleculesHistogramEvery 5000

### Scenario 2: Complex Request
User specifies Temperature and a Pressure Range. The agent should map these correctly and format the pressure list as space-separated values.

In [4]:
request_2 = "Run a methane isotherm in ZIF-8 at 300K with pressures 10kPa, 50kPa, and 100kPa."

output_2 = agent.run(request_2, mock=(GOOGLE_API_KEY is None), output_filename="simulation_case2.input")

print("\n--- Generated File Preview ---\n")
print(output_2)

--- Processing Request: 'Run a methane isotherm in ZIF-8 at 300K with pressures 10kPa, 50kPa, and 100kPa.' ---
[Request ID: 4]
✔ Extracted Entities: {'simulation_type': 'MonteCarlo', 'system': {'framework_name': 'ZIF-8', 'temperature': 300.0, 'pressure_list': [10000.0, 50000.0, 100000.0]}, 'parameters': {}, 'component': {'name': 'methane', 'mole_fraction': 1.0}}
✔ Applied Defaults.
✔ RASPA file generated (1088 chars)
✔ File saved to: simulation_case2.input

--- Generated File Preview ---

SimulationType                MonteCarlo
NumberOfCycles                25000
NumberOfInitializationCycles  5000
PrintEvery                    1000
RestartFile                   no

Forcefield                    UFF
CutOff                        12.0
ChargeMethod                  Ewald
EwaldPrecision                1e-06
RemoveAtomNumberCodeFromLabel yes

Framework 0
FrameworkName ZIF-8
UnitCells 2 2 2
HeliumVoidFraction 0.29
ExternalTemperature 300.0
ExternalPressure 10000.0 50000.0 100000.0

ComputeN

### Scenario 3: Verify Defaults
What happens if the user says something completely vague? The default logic should kick in to ensure the simulation file is still runnable (using MFI_SI and standard settings).

In [ ]:
request_3 = "Just run a standard simulation"

output_3 = agent.run(request_3, mock=(GOOGLE_API_KEY is None))

print("\n--- Generated File Preview ---\n")
print(output_3)

### View Persistent Logs
Check the `raspa_log.json` file to see all request history with timestamps and step-by-step timing.


In [16]:
import json

# Run a fresh request to generate logs
print("=== Running a fresh request with logging ===\n")
request_fresh = "Simulate methane in ZIF-8 at 350K"
output_fresh = agent.run(request_fresh, mock=(GOOGLE_API_KEY is None), output_filename="simulation_fresh.input")

# Now display the logs
print("\n=== PERSISTENT LOG (raspa_log.json) ===\n")
with open("raspa_log.json", "r") as f:
    logs = json.load(f)

# Pretty print the logs
print(json.dumps(logs, indent=2))


=== Running a fresh request with logging ===

--- Processing Request: 'Simulate methane in ZIF-8 at 350K' ---
✔ Extracted Entities: {'simulation_type': 'MonteCarlo', 'system': {'framework_name': 'ZIF-8', 'temperature': 350.0}, 'parameters': {}, 'component': {'name': 'methane', 'mole_fraction': 1.0}}
✔ Applied Defaults.
✔ File saved to: simulation_fresh.input

=== PERSISTENT LOG (raspa_log.json) ===

{
  "history": []
}


In [19]:
import importlib
import sys
import os
import json

# Force reload of both modules to get the latest code
for module_name in ['raspa_logger', 'raspa_agent']:
    if module_name in sys.modules:
        del sys.modules[module_name]

print("✔ Reloaded modules with complete logging\n")

from raspa_agent import RaspaAgent
agent = RaspaAgent(api_key=GOOGLE_API_KEY)

# Run a fresh request to generate logs
print("=== Running a fresh request with logging ===\n")
request_fixed = "Simulate CO2 in HKUST-1 at 298K"
output_fixed = agent.run(request_fixed, mock=(GOOGLE_API_KEY is None), output_filename="simulation_test.input")

print("\n--- Generated File Preview ---\n")
print(output_fixed[:500] + "..." if len(output_fixed) > 500 else output_fixed)

# Now display the logs - check actual log file path
print("\n=== PERSISTENT LOG (raspa_log.json) ===\n")
print(f"Log path being used: {agent.logger.log_path}\n")

with open(agent.logger.log_path, "r") as f:
    logs = json.load(f)

print(json.dumps(logs, indent=2))


✔ Reloaded modules with complete logging

=== Running a fresh request with logging ===

--- Processing Request: 'Simulate CO2 in HKUST-1 at 298K' ---
[Request ID: 2]
✔ Extracted Entities: {'simulation_type': 'MonteCarlo', 'system': {'framework_name': 'HKUST-1', 'temperature': 298.0}, 'parameters': {}, 'component': {'name': 'CO2', 'mole_fraction': 1.0}}
✔ Applied Defaults.
✔ RASPA file generated (1068 chars)
✔ File saved to: simulation_test.input

--- Generated File Preview ---

SimulationType                MonteCarlo
NumberOfCycles                25000
NumberOfInitializationCycles  5000
PrintEvery                    1000
RestartFile                   no

Forcefield                    UFF
CutOff                        12.0
ChargeMethod                  Ewald
EwaldPrecision                1e-06
RemoveAtomNumberCodeFromLabel yes

Framework 0
FrameworkName HKUST-1
UnitCells 2 2 2
HeliumVoidFraction 0.29
ExternalTemperature 298.0
ExternalPressure 100000

ComputeNumberOfMol...

=== PERSISTE